In [1]:
import pandas as pd
import os
import pyperclip
from paths import *

In [2]:
"""
Serum. Untreated - caffeine pathway
Feces. Untreated  - Phenylalanine, tyrosine and tryptophan biosynthesis
"""


'\nSerum. Untreated - caffeine pathway\nFeces. Untreated  - Phenylalanine, tyrosine and tryptophan biosynthesis\n'

In [3]:
"""
data_1_pubchem_total is directly taken from MetaboAnalyst. 
After giving the names, it returned the associated pubchem ids of the targeted compounds
"""
data_1_pubchem_total = pd.DataFrame([
                               ("2-Methylbutyric acid", "8314"), 
                               ("Acetic acid", "176"), 
                               ("Butyric acid", "264"),
                                ("Hexanoic acid", "8892"),
                                ("Isobutyric acid", "6590"),                               
                               ("Isovaleric acid", "10430"), 
                               ("Propionic acid", "1032"),
                                ("Valeric acid", "7991")
                              ], 
                              columns=["analyte_name", "PUBCHEM"])


In [68]:
sample = "feces"
sheet_index = 2

if sample == "feces":
    global_data_path = GLOBAL_STOOL_DATA_FILENAME
else:
    global_data_path = GLOBAL_SERUM_DATA_FILENAME

pval_thresh = 0.05
sheet_name_list = ["Without outlier", "Without outlier-MS treated", "Without outlier-MS not treated",
                  "With outlier", "With outlier-MStreated", "With outlier-MS not treated"]



file_path1 = os.path.join(OUTPUT_PATH, "GLM_result_targeted_compounds_{}_sample.xlsx".format(sample))
file_path2 = os.path.join(OUTPUT_PATH, "GLM_result_global_compounds_{}_sample.xlsx".format(sample))


In [69]:

data_1 = pd.read_excel(file_path1, engine='openpyxl', sheet_name = sheet_name_list[sheet_index])
data_1 = data_1[(data_1["model_converged_flag"]==1) & (data_1["pvalue"] < pval_thresh)]


data_2 = pd.read_excel(file_path2, engine='openpyxl', sheet_name = sheet_name_list[sheet_index])
data_2 = data_2[(data_2["model_converged_flag"]==1) & (data_2["pvalue"] < pval_thresh)]

global_data = pd.read_excel(os.path.join(DATA_ROOT_PATH, global_data_path), engine='openpyxl', sheet_name="Chemical Annotation")



In [70]:
data_1_pubchem = pd.merge(data_1, data_1_pubchem_total, on="analyte_name")[["analyte_name", "PUBCHEM"]]


In [71]:
data_2_pubchem = pd.merge(data_2, global_data, left_on="chem_id", right_on="CHEM_ID")[["analyte_name", "PUBCHEM"]]
data_2_pubchem.PUBCHEM = data_2_pubchem.PUBCHEM.astype("str")
data_2_pubchem.PUBCHEM = data_2_pubchem.PUBCHEM.apply(lambda x:x.split(","))
data_2_pubchem = data_2_pubchem.explode("PUBCHEM").drop_duplicates(subset="analyte_name")


In [72]:
data_pubchem = pd.concat([data_1_pubchem, data_2_pubchem], ignore_index=True).drop_duplicates(subset="PUBCHEM")
data_pubchem.analyte_name = data_pubchem.analyte_name.apply(lambda x:x.split("(")[0])
data_pubchem.analyte_name = data_pubchem.analyte_name.apply(lambda x:x.split("*")[0])


In [73]:
data_pubchem

,analyte_name,PUBCHEM
0,2-Methylbutyric acid,8314
1,Acetic acid,176
2,Isovaleric acid,10430
3,Propionic acid,1032
4,S-1-pyrroline-5-carboxylate,11966181
...,...,...
311,N-methylproline,643474
312,beta-cryptoxanthin,5281235
313,N-acetyl-1-methylhistidine,53859791
314,fructosyllysine,9839580


In [74]:
analytes = list(data_pubchem["PUBCHEM"].unique())

# analytes = list(data_pubchem["analyte_name"].unique())



In [75]:
analytes_text = "\n".join(analytes)


In [76]:
pyperclip.copy(analytes_text)


In [77]:
len(analytes)

299